In [126]:
%%capture
import pandas as pd
import requests as rq
import pandas_profiling
import calendar as cl
import os
import time
import json
import re
import numpy as np
import glob

In [127]:
#generamos el rango de fechas para realizar la busqueda

Dates = pd.date_range(start='2022-12-01', end='2022-12-31', freq='D', normalize=False)

Dates_Series = Dates.to_series().dt.strftime('%Y-%m-%d')

In [124]:
try:
  os.makedirs("json\Raw_period_date=2022-12")
except:
  pass

try:
  os.makedirs("profiling")
except:
  pass

try:
  os.makedirs("json\Raw_parquet_period=2022-12")
except:
  pass

try:
  os.makedirs("json\master_parquet_period=2022-12")
except:
  pass

In [125]:
##data hanlder via api 
for date_find in Dates_Series :
  # print(date_find)


  response = rq.get(f"http://api.tvmaze.com/schedule/web?date={date_find}")

  
  # valida el status
  if response.status_code == 200:
      # The request was successful, so parse the response data
      data = json.loads(response.content)
      # Save the data to a JSON file
      with open(f"json/Raw_period_date=2022-12/{Dates_Series[date_find]}.json", "w") as f:
          json.dump(data, f, indent=4)
          f.close()
          time.sleep(0.10)
  else:
      # The request failed, so print the error message
      print(response.status_code)
      print(response.content)


In [143]:
%%capture 
#data profilling generator

def return_profilling_per_date(day):

    df = pd.read_json(f"../json/Raw_period_date=2022-12/{day}.json")
    profiling = df.profile_report(title=f'Profiling {day}')
    profiling.to_file(f'../profiling/profiling_{day}.html')

    return "ok"

for date_find in Dates_Series :
    return_profilling_per_date(date_find)

In [ ]:
#limpieza para columna rating 0 para none 

#df = pd.read_json(f"period_date=2022-12/2022-12-01.json")

def clean_rating(df,column):
    
   df[column] = df[column].apply(lambda x: x.get("average"))

   df[column] = df[column].replace(np.NAN, 0)
    
   return df

